# Result Analysis

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

import numpy as np
from scipy import stats

import pandas as pd
import pickle
import torch
from itertools import combinations
from pathlib import Path
from tqdm import tqdm_notebook as tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler

from utils.metrics import BinaryAvgMetrics
from utils.plots import *
from args import args
vars(args)

{'workdir': PosixPath('../data/workdir'),
 'figdir': PosixPath('../data/figures'),
 'raw_csv': PosixPath('../data/raw_dataset.csv'),
 'proc_csv': PosixPath('../data/proc_dataset.csv'),
 'imminent_adm_cols': ['hadm_id', 'imminent_adm_label'],
 'prolonged_stay_cols': ['hadm_id', 'prolonged_stay_label'],
 'cols': ['hadm_id',
  'imminent_adm_label',
  'prolonged_stay_label',
  'processed_note',
  'charttime',
  'intime'],
 'dates': ['charttime', 'intime'],
 'ia_thresh': {'lr': 0.45, 'rf': 0.27, 'gbm': 0.435, 'mlp': 0.2},
 'ps_thresh': {'lr': 0.39, 'rf': 0.36, 'gbm': 0.324, 'mlp': 0.27}}

## 100 Run Performance Results

In [3]:
models = ['lr', 'rf', 'gbm']

ia_bams = {}
ps_bams = {}

for model in models:
  with open(args.workdir/model/'imminent_adm_preds.pkl', 'rb') as f:
    targs = pickle.load(f)
    preds = pickle.load(f)
    probs = pickle.load(f)
  ia_bams[model] = BinaryAvgMetrics(targs, preds, probs)
    
  with open(args.workdir/model/'prolonged_stay_preds.pkl', 'rb') as f:
    targs = pickle.load(f)
    preds = pickle.load(f)
    probs = pickle.load(f)    
  ps_bams[model] = BinaryAvgMetrics(targs, preds, probs)    

In [4]:
ia_metrics = {}
ps_metrics = {}

for key in ia_bams.keys():
  ia_metrics[key] = []
  ps_metrics[key] = []
  for i in range(len(ia_bams[key].get_avg_metrics())):
    ia_metrics[key].append(ia_bams[key].get_avg_metrics().iloc[i]['Value'])
  for i in range(len(ps_bams[key].get_avg_metrics())):
    ps_metrics[key].append(ps_bams[key].get_avg_metrics().iloc[i]['Value'])    

ia_metrics = pd.DataFrame(ia_metrics, index=['sensitivity', 'specificity', 'ppv', 'auroc', 'npv', 'f1'])
ps_metrics = pd.DataFrame(ps_metrics, index=['sensitivity', 'specificity', 'ppv', 'auroc', 'npv', 'f1'])

In [10]:
ps_bams['gbm'].get_avg_metrics(conf=0.95)

,Mean,Lower,Upper
sensitivity,67.9,67.3,68.5
specificity,39.5,39.1,39.9
ppv,40.6,39.9,41.3
auroc,55.2,54.7,55.6
npv,66.9,66.1,67.7
f1,50.7,50.1,51.3


In [6]:
ps_metrics

,lr,rf,gbm,mlp
sensitivity,82.7,69.1,71.9,71.3
specificity,24.2,39.9,35.4,37.4
ppv,39.9,41.1,40.3,41.0
auroc,55.5,56.3,55.3,56.2
npv,69.6,68.0,67.4,68.2
f1,53.7,51.4,51.6,51.9


## Student-t Test

In [7]:
models = list(ia_bams.keys())
metrics = list(ia_metrics.index)

In [8]:
def do_ttest(bams, model1, model2, metric):  
  if metric == 'sensitivity':
    x1 = bams[model1].sensitivities()
    x2 = bams[model2].sensitivities()
  elif metric == 'specificity':
    x1 = bams[model1].specificities()
    x2 = bams[model2].specificities()
  elif metric == 'ppv':
    x1 = bams[model1].ppvs()
    x2 = bams[model2].ppvs()
  elif metric == 'auroc':
    x1 = bams[model1].aurocs()
    x2 = bams[model2].aurocs()
  elif metric == 'npv':
    x1 = bams[model1].npvs()
    x2 = bams[model2].npvs()
  elif metric == 'f1':    
    x1 = bams[model1].f1s()
    x2 = bams[model2].f1s()

  t, p = stats.ttest_ind(x1, x2)
  return t, p

In [11]:
ia_ttests = {}

for m1, m2 in combinations(models, 2):  
  ia_ttests[f'{m1}-{m2}'] = {}
  for metric in metrics:
    ia_ttests[f'{m1}-{m2}'][metric] = do_ttest(ia_bams, m1, m2, metric)

In [9]:
ia_ttests = pd.DataFrame(ia_ttests)
ia_ttests

,lr-rf,lr-gbm,lr-mlp,rf-gbm,rf-mlp,gbm-mlp
auroc,"(-7.7451541928851775, 4.836516995686219e-13)","(-8.265869341137545, 1.9793917965523378e-14)","(-5.913832276312028, 1.4448892210423221e-08)","(-0.7963450632222485, 0.42678527078590933)","(1.7767417752067243, 0.0771461533694063)","(2.5000550426675607, 0.013229397821328327)"
f1,"(-13.89259619760442, 4.4468872632205e-31)","(-13.712645622349955, 1.5871689964687176e-30)","(-9.738157068213015, 1.4727586081772186e-18)","(0.5033655531746059, 0.6152668087515873)","(3.963876278335994, 0.00010293292087679879)","(3.565098804431242, 0.0004559422617680369)"
npv,"(-1.0517296706690993, 0.2942054660560996)","(-1.0176017655306844, 0.310109050179264)","(0.7199946199596783, 0.47237736099338334)","(0.02193064360802606, 0.9825253654405779)","(1.8824477350818785, 0.06124194952108758)","(1.833805765631917, 0.06818393843722381)"
ppv,"(-15.89810013872224, 3.2155254168682093e-37)","(-15.99854196157006, 1.5906084459756007e-37)","(-11.609813715156726, 4.182813384853509e-24)","(0.582729231178791, 0.5607386350848496)","(3.3070110604813525, 0.0011197712503367857)","(2.881535938008341, 0.004393865656583489)"
sensitivity,"(4.999297358269423, 1.2634197125285202e-06)","(4.623214869345008, 6.80768136354464e-06)","(6.354780615008556, 1.406329721437722e-09)","(-0.16802789210935817, 0.866732840866424)","(1.8503438683672044, 0.06575392241155188)","(1.916222394257942, 0.05677704089469465)"
specificity,"(-16.770036865303737, 7.2930676438118314e-40)","(-15.570375747392129, 3.208801270403557e-36)","(-12.675706530730535, 2.3956312833456183e-27)","(0.5671969467662671, 0.5712228678796696)","(2.5119867748365676, 0.012804002351635481)","(1.9156725892759001, 0.05684747686316934)"


In [10]:
ps_ttests = {}

for m1, m2 in combinations(models, 2):  
  ps_ttests[f'{m1}-{m2}'] = {}
  for metric in metrics:
    ps_ttests[f'{m1}-{m2}'][metric] = do_ttest(ps_bams, m1, m2, metric)

ps_ttests = pd.DataFrame(ps_ttests)
ps_ttests

,lr-rf,lr-gbm,lr-mlp,rf-gbm,rf-mlp,gbm-mlp
auroc,"(-2.1260637847824064, 0.034736656458580914)","(0.5152167599158811, 0.6069762865464206)","(-2.0973364861139223, 0.037233111879323405)","(2.763893075438058, 0.0062506645311561455)","(0.25207071159648764, 0.8012478673136824)","(-2.8244973599556613, 0.005220011935923832)"
f1,"(5.4856640256470355, 1.2487338411200422e-07)","(5.2096380923278085, 4.729755106066353e-07)","(4.350736192917731, 2.171454634947159e-05)","(-0.3067134095702049, 0.7593838290578625)","(-1.06985515573178, 0.2859869819316188)","(-0.7722012508757278, 0.4409159553251232)"
npv,"(2.224168768899269, 0.027266742564268382)","(3.044453150907716, 0.002647813213641131)","(2.0058042690502185, 0.04623827378315275)","(0.9588868994358638, 0.33878514329922205)","(-0.40035227342284235, 0.6893286916200384)","(-1.4348310481290438, 0.15291287893305713)"
ppv,"(-2.681992593768923, 0.00793652965546063)","(-1.035675266480739, 0.3016168886557689)","(-2.267090198167265, 0.024465142561219424)","(1.602092840652466, 0.11072921903994894)","(0.36225521176391334, 0.7175476714351372)","(-1.2161521566037623, 0.2253748260676345)"
sensitivity,"(29.457780811012544, 2.6836891232103287e-74)","(26.540928934886537, 3.8957168375885415e-67)","(26.142551944740777, 4.01596927399462e-66)","(-5.87326909149356, 1.780554081470574e-08)","(-4.5206781026411464, 1.0597455513793557e-05)","(1.1961868756842087, 0.23305410085009)"
specificity,"(-46.74594148317807, 6.352849341508821e-109)","(-38.30769439157174, 1.6508800823375676e-93)","(-34.19247387228388, 5.146320540472835e-85)","(13.066908849914673, 1.5199730299414182e-28)","(5.778100740270058, 2.895977746353955e-08)","(-5.146568774048572, 6.36911556999194e-07)"


## Debug PPV

In [ ]:
model = 'lr'

In [ ]:
ia, ps = ia_bams[model], ps_bams[model]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plot_confusion_matrix(ax, ia.cm_avg, classes=['Negative', 'Positive'], normalize=False, title='Confusion matrix')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plot_confusion_matrix(ax, ps.cm_avg, classes=['Negative', 'Positive'], normalize=False, title='Confusion matrix')

In [ ]:
sensitivity = tps/(tps + fns)

In [ ]:
sensitivity

In [ ]:
ppv = tps/(tps + fps)

In [ ]:
ppv

## Mean AUC

In [ ]:
model = 'mlp'
ia_bams[model].get_avg_metrics(conf=0.5)

In [ ]:
model = 'mlp'
ps_bams[model].get_avg_metrics(conf=0.5)

In [ ]:
def get_mean_tprs(bams, base_fpr):
  mean_tprs = {}  
  for model, bam in bams.items():
    tprs = []  
    for i, (targs, probs) in enumerate(zip(bam.targs, bam.probs)):
      fpr, tpr, _ = roc_curve(targs, probs)
      tpr = interp(base_fpr, fpr, tpr)
      tpr[0] = 0.0
      tprs.append(tpr)

    tprs = np.array(tprs)
    mean_tprs[model] = tprs.mean(axis=0)
    
  return mean_tprs

In [ ]:
base_fpr = np.linspace(0, 1, 100)
mean_tprs = get_mean_tprs(ia_bams, base_fpr)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
for i, (model, mean_tpr) in enumerate(mean_tprs.items()):
  ax.plot(base_fpr, mean_tpr)
ax.plot([0, 1], [0, 1], linestyle=':')  
ax.grid(b=True, which='major', color='#d3d3d3', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d3d3d3', linewidth=0.5)
ax.set_ylabel('Sensitivity')
ax.set_xlabel('1 - Specificity')
ax.legend(['Logistic Regression', 'Random Forests', 'Gradient Boosting Machines', 'Multilayer Perceptron'])
fig.savefig(args.figdir/'ia_mean_roc.tif', dpi=300)

In [ ]:
base_fpr = np.linspace(0, 1, 100)
mean_tprs = get_mean_tprs(ps_bams, base_fpr)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
for i, (model, mean_tpr) in enumerate(mean_tprs.items()):
  ax.plot(base_fpr, mean_tpr)
ax.plot([0, 1], [0, 1], linestyle=':')  
ax.grid(b=True, which='major', color='#d3d3d3', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d3d3d3', linewidth=0.5)
ax.set_ylabel('Sensitivity')
ax.set_xlabel('1 - Specificity')
ax.legend(['Logistic Regression', 'Random Forests', 'Gradient Boosting Machines', 'Multilayer Perceptron'])
fig.savefig(args.figdir/'ps_mean_roc.tif', dpi=300)

## Probability Plots

In [ ]:
df = pd.read_csv(args.proc_csv, usecols=args.cols, parse_dates=args.dates)
df['relative_charttime'] = (df['charttime'] - df['intime'])

imminent_df = df.loc[(df['imminent_label'] != -1)][['scispacy_note', 'imminent_label', 'relative_charttime']].reset_index()
discharge_df = df[['scispacy_note', 'discharge_label', 'relative_charttime']].reset_index()

In [ ]:
interval_hours=12
starting_day = -20
ending_day = -1

### Imminent ICU Admission

In [ ]:
model = 'lr'
with open(args.workdir/model/'imminent_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary)
imminent_df[f'{model}_prob'] = clf.predict_proba(vectorizer.fit_transform(imminent_df['scispacy_note']))[:, 1]

model = 'rf'
with open(args.workdir/model/'imminent_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary)
imminent_df[f'{model}_prob'] = clf.predict_proba(vectorizer.fit_transform(imminent_df['scispacy_note']))[:, 1]

model = 'gbm'
with open(args.workdir/model/'imminent_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary)
imminent_df[f'{model}_prob'] = clf.predict_proba(vectorizer.fit_transform(imminent_df['scispacy_note']))[:, 1]

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 15))
plot_prob(ax[0], imminent_df, 'lr', args.imminent_threshold['lr'], starting_day, ending_day, interval_hours)
plot_prob(ax[1], imminent_df, 'rf', args.imminent_threshold['rf'], starting_day, ending_day, interval_hours)
plot_prob(ax[2], imminent_df, 'gbm', args.imminent_threshold['gbm'], starting_day, ending_day, interval_hours)

fig.text(0.5, 0.08, 'Time to ICU (days)', ha='center')
fig.text(0.08, 0.5, 'Probability', va='center', rotation='vertical')

### ICU Discharge

In [ ]:
model = 'lr'
with open(args.workdir/model/'discharge_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)

vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary)
discharge_df[f'{model}_prob'] = clf.predict_proba(vectorizer.fit_transform(discharge_df['scispacy_note']))[:, 1]

model = 'rf'
with open(args.workdir/model/'discharge_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary)
discharge_df[f'{model}_prob'] = clf.predict_proba(vectorizer.fit_transform(discharge_df['scispacy_note']))[:, 1]

model = 'gbm'
with open(args.workdir/model/'discharge_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary)
discharge_df[f'{model}_prob'] = clf.predict_proba(vectorizer.fit_transform(discharge_df['scispacy_note']))[:, 1]

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 15))
plot_prob(ax[0], discharge_df, 'lr', args.discharge_threshold['lr'], starting_day, ending_day, interval_hours)
plot_prob(ax[1], discharge_df, 'rf', args.discharge_threshold['rf'], starting_day, ending_day, interval_hours)
plot_prob(ax[2], discharge_df, 'gbm', args.discharge_threshold['gbm'], starting_day, ending_day, interval_hours)

fig.text(0.5, 0.08, 'Time to ICU (days)', ha='center')
fig.text(0.08, 0.5, 'Probability', va='center', rotation='vertical')

## Word Clouds

### Imminent ICU Admission

In [ ]:
scores = {}
feature_names = {}

model = 'lr'
with open(args.workdir/model/'imminent_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)

feature_names[model] = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary).get_feature_names()
scores[model] = clf.coef_[0]/clf.coef_[0].sum()

model = 'rf'
with open(args.workdir/model/'imminent_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)

feature_names[model] = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary).get_feature_names()
scores[model] = clf.feature_importances_/clf.feature_importances_.sum()

model = 'gbm'
with open(args.workdir/model/'imminent_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)

feature_names[model] = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary).get_feature_names()
scores[model] = clf.feature_importances_/clf.feature_importances_.sum()

In [ ]:
model = 'gbm'
fig, ax = plt.subplots(1, 2, figsize=(15, 10))
neg, pos = get_wordcloud(feature_names[model], scores[model], n_words=25)
ax[0].imshow(neg)
ax[0].axis('off')
ax[1].imshow(pos)
ax[1].axis('off')

### ICU Discharge

In [ ]:
scores = {}
feature_names = {}

model = 'lr'
with open(args.workdir/model/'discharge_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)

feature_names[model] = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary).get_feature_names()
scores[model] = clf.coef_[0]/clf.coef_[0].sum()

model = 'rf'
with open(args.workdir/model/'discharge_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)

feature_names[model] = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary).get_feature_names()
scores[model] = clf.feature_importances_/clf.feature_importances_.sum()

model = 'gbm'
with open(args.workdir/model/'discharge_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)

feature_names[model] = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True, ngram_range=(2,2), vocabulary=vocabulary).get_feature_names()
scores[model] = clf.feature_importances_/clf.feature_importances_.sum()

In [ ]:
model = 'gbm'
fig, ax = plt.subplots(1, 2, figsize=(15, 12))
neg, pos = get_wordcloud(feature_names[model], scores[model], n_words=50)
ax[0].imshow(neg)
ax[0].axis('off')
ax[1].imshow(pos)
ax[1].axis('off')

In [ ]:
# fig.savefig(args.figdir/f'prob.tif', dpi=300)